In [2]:
%load_ext autoreload
%autoreload 2

## Dataset & Dataloader

In [3]:
import os
os.chdir("../")

from omegaconf import OmegaConf

from neuralfp.data.datasets import MusicSegmentDataset, collate_data
from neuralfp.utils.common import load_dataset

In [4]:
config = "configs/train.yaml"
config = OmegaConf.load(config)

dataset = MusicSegmentDataset(config["dataset"]["train"])

Loading IR: 100%|██████████| 345/345 [00:03<00:00, 110.66it/s]


In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    collate_fn=collate_data,
    shuffle=False,
    **config["dataset"]["loaders"],
)

In [6]:
# import tqdm

# for batch in tqdm.tqdm(dataloader):
#     features, targets = batch
#     print(features.shape, targets.shape)

## Model

In [7]:
from neuralfp.model.neuralfp import NeuralAudioFingerprinter

In [8]:
config = "configs/train.yaml"
config = OmegaConf.load(config)

model = NeuralAudioFingerprinter(**config["model"]["neuralfp"])

In [9]:
import tqdm
import torch

for batch in tqdm.tqdm(dataloader):
    features, targets = batch
    print("features", features.shape)
    xs = torch.stack([features, targets], dim=0)
    print("xs", xs.shape)

    xs = torch.flatten(xs, 0, 1)
    out = model(xs)
    print("out", out.shape)


  0%|          | 0/5000 [00:00<?, ?it/s]/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


features torch.Size([116, 256, 32])
xs torch.Size([2, 116, 256, 32])


  0%|          | 1/5000 [00:26<36:36:52, 26.37s/it]

out torch.Size([232, 128])
features torch.Size([116, 256, 32])
xs torch.Size([2, 116, 256, 32])


  0%|          | 1/5000 [00:29<40:56:09, 29.48s/it]


KeyboardInterrupt: 

## Loss function

In [10]:
from neuralfp.criterion.contrastive_loss import NTxentLoss

criterion = NTxentLoss()

In [11]:
import tqdm
import torch

for batch in tqdm.tqdm(dataloader):
    features, targets = batch
    xs = torch.stack([features, targets], dim=0)
    xs = torch.flatten(xs, 0, 1)
    out = model(xs)
    n_anchors = out.shape[0] // 2
    print("n_anchors", n_anchors)
    loss = criterion(
        out[:n_anchors, :], out[n_anchors:, :], n_anchors
    )
    print("loss", loss)

  0%|          | 0/5000 [00:00<?, ?it/s]

/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/huynd/anaconda3/envs/audiofp/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


n_anchors 116


  0%|          | 1/5000 [00:10<14:44:08, 10.61s/it]

loss tensor(10.8773, grad_fn=<AddBackward0>)


...

loss tensor(10.8949, grad_fn=<AddBackward0>)
n_anchors 116


  0%|          | 13/5000 [00:51<4:55:32,  3.56s/it]

loss tensor(10.8878, grad_fn=<AddBackward0>)


  0%|          | 13/5000 [00:53<5:39:51,  4.09s/it]


KeyboardInterrupt: 